In [1]:
import pandas as pd
import numpy as np

import statistics

import re

import nltk

import kiwipiepy
from kiwipiepy import Kiwi # 형태소 분석
from kiwipiepy.utils import Stopwords # kiwi 불용어

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm
import time

In [2]:
def Rep_text(text): # 단어 통일 시키기
    text = str(text)
    pattern = '(\n){1,}' 
    text = re.sub(pattern=pattern, repl='\n', string=text) # \n가 1번이상 사용된 경우 \n로 바꾸기
    return text

# data load

In [3]:
# 파일 열기
df = pd.read_csv('../data/pre_data.csv')

df['preprocessed_best_comment'] = df['preprocessed_best_comment'].apply(lambda x: Rep_text(x)) # \n 반복되는 것 제거
df['preprocessed_comment'] = df['preprocessed_comment'].apply(lambda x: Rep_text(x)) # \n 반복되는 것 제거

# dataset 정의
- 각 댓글별로 row로 만들기
- 1.5일 소요  
- 각 웹툰의 에피소드의 댓글을 분리할 때, 해당 에피소드의 sentiment 부여
- 3296688개의 row여서 랜덤 샘플하여 데이터 줄이기

In [6]:
def Commend(best,com):
    return df[best]+'\n'+df[com] # best와 일반 댓글 합치기

df['total_comment'] = Commend('preprocessed_best_comment','preprocessed_comment')

In [16]:
df_set = pd.DataFrame(columns=['comment','sentiment'])

for i in tqdm(range(len(df))):
    sentiment = df['sentiment'][i]
    for sen in df['total_comment'][i].split('\n'): # 한 user의 댓글로 나눠서 별도의 row로 저장
        df_set.loc[len(df_set)] = [sen,sentiment]

100%|████████████████████████████████████████████████████████████████████████████| 4949/4949 [6:10:15<00:00,  4.49s/it]


In [ ]:
df_set = df_set.dropna(how = 'any') # Null 값이 존재하는 행 제거
df_set['comment'] = df_set['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글만 남기기

# df_set.to_csv('../data/comment_data.csv', index=False)

In [28]:
# 기존 데이터 3296688 rows×2 columns
# df = pd.read_csv('../data/comment_data.csv')

# 데이터 양 줄이기. 긍정:부정=1:5 비율
df_po = df.query('sentiment == 1').sample(n=5000, random_state=1) # 2606508->5000
df_ne = df.query('sentiment == 0').sample(n=25000, random_state=1) # 676330->25000
df_total = pd.concat([df_po,df_ne])

df_total = df_total.dropna(how = 'any') # Null 값이 존재하는 행 제거

In [29]:
# kiwi활용 토큰화
kiwi_stop = Stopwords() # kiwi의 불용어로만 처리
kiwi = kiwipiepy.Kiwi()

def extract_nouns(text): # kiwi사용하여 토큰화
    tt = ""
    for token in kiwi.tokenize(text, normalize_coda=True, stopwords=kiwi_stop):
        if (token.tag in {'NNG', 'NNP'}) & (len(token.form)>1): # kiwi 명사 추출.
            tt+=f'{token.form} ' # 하나로 합치기
    return tt

df_total['pre_comment'] = df_total['comment'].apply(lambda x: extract_nouns(x))
# df_total.to_csv('../data/senti_data.csv',index=False)

# TF-IDF 만들기

In [19]:
df_total = pd.read_csv('../data/senti_data.csv')

,comment,sentiment,pre_comment
0,근데 란 기술 쓰는 거 보면 죄다 스페인어 이름들이고 딸 아들들도 하나같이 다 서양...,1,기술 스페인어 이름 아들 서양 주인 에드 동양식 복장
1,이랬던 자림이가 어쩌다가,1,자림이
2,소연이 로우킥이 원주민 공포 만화 얄라 뽕 따지 급인데,1,소연 로우 원주민 공포 만화 얄라
3,강효문이 솔까 한 유현 보다 더 잘생겼는데,1,강효 유현
4,와느무잼따,1,와느무잼따
...,...,...,...
25811,작가님 진짜 제발 부탁인데요 김신 씨 죽이지 말아 주세요 우리 애 너무 불쌍하잖아요...,0,작가 부탁 김신 아버지 태양
25812,웃긴 건 저기보다 남쪽인 남만 애들 사는 땅은 고산지대라고 사계절 봄 날씨임,0,남쪽 남만 고산 지대 사계절 날씨
25813,그냥 급도 벌레 취급이네 밸붕,0,벌레 취급
25814,갑자기 얼굴을 저렇게 들 이 미시 면 감사합니다,0,얼굴 감사


In [20]:
df_total = df_total.dropna(how = 'any') # Null 값이 존재하는 행 제거

tfidf = TfidfVectorizer() #TF-IDF 만들기
tfidf_mat = tfidf.fit_transform(df_total['pre_comment']) # 설명부분
tfidf_mat # 25816개의 댓글, 각 댓글은 16500개의 유니크한 단어

<25816x16500 sparse matrix of type '<class 'numpy.float64'>'
	with 89182 stored elements in Compressed Sparse Row format>

In [5]:
# 형태 변환하기
x = tfidf_mat.toarray()
y = df_total.sentiment.to_numpy()

In [6]:
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)

# 감정분석
- keras 사용

In [ ]:
# !pip install -q tensorflow

In [8]:
import tensorflow as tf

# 모델 구성
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, activation='sigmoid')) # layer 1층만 사용

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train, epochs=20) # 모델 학습

Epoch 1/20
565/565 [==============================] - 2s 2ms/step - loss: 0.6041 - accuracy: 0.8283
Epoch 2/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4984 - accuracy: 0.8334: 0s - loss: 0.5056 
Epoch 3/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4562 - accuracy: 0.8334
Epoch 4/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4387 - accuracy: 0.8334
Epoch 5/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4300 - accuracy: 0.8334
Epoch 6/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4242 - accuracy: 0.8334
Epoch 7/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4191 - accuracy: 0.8334
Epoch 8/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4140 - accuracy: 0.8334
Epoch 9/20
565/565 [==============================] - 1s 2ms/step - loss: 0.4092 - accuracy: 0.8334
Epoch 10/20
565/565 [==============================] - 1s 2ms/step - loss: 0.404

In [12]:
# 모델 저장
# model.save('comments_sem.krs')

INFO:tensorflow:Assets written to: comments_sem.krs\assets


In [9]:
model = tf.keras.models.load_model('comments_sem.krs') # 저장했던 모델 불러오기

In [10]:
model.evaluate(x_test, y_test) # 모델 평가

243/243 [==============================] - 1s 2ms/step - loss: 0.4395 - accuracy: 0.8333


[0.43948912620544434, 0.8333117961883545]

In [11]:
w, b = model.weights # 가중치 출력하기

In [15]:
word_sent = pd.DataFrame({'토큰': tfidf.get_feature_names_out(), '가중치': w.numpy().flat}) # 토큰과 가중치를 데이터프레임으로 변환

In [16]:
# 상대적으로 부정 문장에서 많이 나오는 단어
word_sent.sort_values('가중치').head(20)

,토큰,가중치
12101,제갈량,-2.709445
12009,정사,-2.313717
12300,조조,-2.242916
1179,관우,-2.140237
14467,테러,-1.968266
7425,손권,-1.908531
5873,본인,-1.814535
7681,순욱,-1.694482
11515,장료,-1.625467
9078,억지,-1.620497


In [17]:
# 상대적으로 긍정 문장에서 많이 나오는 단어
word_sent.sort_values('가중치').tail(20)

,토큰,가중치
2781,단어,0.749894
5380,배대,0.752828
2923,대방,0.752890
4940,미영,0.757898
5153,박윤수,0.762431
8096,시현,0.765349
11549,장애인,0.777504
4000,리퍼,0.780392
6433,사기,0.791212
9525,영준이,0.792739
